In [91]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [306]:
data = pd.read_csv('C:/Jupyter/TESIS/instancias/real/testInstance_Amtrak.csv', delimiter=';')
data.drop(columns=['Unnamed: 0'], inplace=True)

In [307]:
start_estation = 'NYP'

origin = data['Origin'].unique().tolist()
destin = data['Destination'].unique().tolist()
periodo = sorted(data['DBD'].unique().tolist(), reverse=True)

stations = destin + [i for i in origin if i not in destin]

anonymo_stat = {i: data[data['Market'] == start_estation + '-' + i]['NbLegs'].unique()[0] + 1 if i != start_estation else 1 for i in stations }
data.replace(anonymo_stat, inplace=True)
origin_cor = sorted(data['Origin'].unique().tolist())
destin_cor = sorted(data['Destination'].unique().tolist())

In [297]:
data['Market'] = data['Origin'].astype('str') +'-'+ data['Destination'].astype('str')
oridest = data['Market'].unique().tolist()
oridest = sorted([(int(i.split('-')[0]), int(i.split('-')[1])) for i in oridest])

jerar_comer_class = {'Y': '1-1','YA': '1-2','YB': '1-3','YC': '1-4','YD': '1-5','YE': '1-6','YF': '1-7','YG': '1-8','JY': '2-1','JA': '2-2','JB': '2-3','JC': '2-4','JD': '2-5','JE': '2-6','JF': '2-7','JG': '2-8'}
data['CommercialClass'].replace(jerar_comer_class, inplace=True)

data[['vagon','clase']] = data['CommercialClass'].str.split('-', expand=True).astype('int')
vagones = data['vagon'].unique().tolist()
clases = {v: sorted(data[data['vagon']==v]['clase'].unique().tolist()) for v in vagones}

In [298]:
demanda = pd.pivot_table(data, values=['Bookings'], index=['Origin','Destination','vagon','clase', 'DBD'], aggfunc={'Bookings':'sum'})
demanda.reset_index(inplace=True)

preco = pd.pivot_table(data, values=['Revenue'], index=['Origin','Destination','vagon','clase'], aggfunc={'Revenue':'max'})
preco.reset_index(inplace=True)

In [299]:
for i,j in oridest:
    for v in vagones:
        for k in clases[v]:
                if preco.loc[(preco.Origin == i) & (preco.Destination == j) & (preco.vagon == v) & (preco.clase == k)]['Revenue'].count() == 0:
                    a = {'Origin':i,'Destination':j,'vagon':v,'clase':k,'Revenue':0}
                    preco.loc[len(preco)] = a

for i,j in oridest:
    for v in vagones:
        for k in clases[v]:
                for t in periodo:
                    if demanda.loc[(demanda.Origin == i) & (demanda.Destination == j) & (demanda.vagon == v) & (demanda.clase == k) & (demanda.DBD == t)]['Bookings'].count() == 0:
                        a = {'Origin':i,'Destination':j,'vagon':v,'clase':k, 'DBD':t,'Bookings':0}
                        demanda.loc[len(demanda)] = a

In [300]:
preco = preco.sort_values(by=['Origin', 'Destination', 'vagon', 'Revenue'], ascending=[True, True, True, False])

dem_cor = demanda.set_index(['Origin', 'Destination', 'vagon', 'clase', 'DBD'])['Bookings'].to_dict()
preco_cor = preco.set_index(['Origin', 'Destination', 'vagon', 'clase'])['Revenue'].to_dict()

In [301]:
# def mode_clase(x):
#     b = x[0]
#     return b

# def sub_clase(x):
#     if len(x)>=2:
#         c = x[1]
#     else:
#         c = ''
#     return c

# data['vagon'] = data['CommercialClass'].apply(mode_clase)
# vagones = data['vagon'].unique().tolist()
# data['vagon'].replace({j:i+1 for i,j in enumerate(vagones)}, inplace= True)

# data['sub_clas'] = data['CommercialClass'].apply(sub_clase)
# sub_clase = sorted(data['sub_clas'].unique().tolist())
# data['sub_clas'].replace({j:i+1 for i,j in enumerate(sub_clase)}, inplace= True)

# vagones_1 = data['vagon'].unique().tolist()
# clases = data['sub_clas'].unique().tolist()